[![Labellerr](https://storage.googleapis.com/labellerr-cdn/%200%20Labellerr%20template/notebook.webp)](https://www.labellerr.com)

# **Research Multi-Agent AI**

---

[![labellerr](https://img.shields.io/badge/Labellerr-BLOG-black.svg)](https://www.labellerr.com/blog/<BLOG_NAME>)
[![Youtube](https://img.shields.io/badge/Labellerr-YouTube-b31b1b.svg)](https://www.youtube.com/@Labellerr)
[![Github](https://img.shields.io/badge/Labellerr-GitHub-green.svg)](https://github.com/Labellerr/Hands-On-Learning-in-Computer-Vision)

## 🎯 Objective
This notebook demonstrates how to build and run a **multi-agent research system** using two frameworks:
1. **CrewAI** – for orchestrating research, analysis, and reporting agents.  
2. **SmolAgents** – for lightweight agent coordination with specialized tools.

The system integrates with APIs (Hugging Face, OpenAI) to perform:
- Sentiment Analysis  
- Entity Extraction  
- Text Summarization  
- Web Research  

By the end, you’ll see how multiple agents can collaborate to generate comprehensive research reports automatically.


## 🛠️ Step 1: Import Required Libraries
We import all dependencies, including:
- **crewai** and **smolagents** for multi-agent orchestration
- **requests** for API calls
- **dotenv** for managing API keys securely
- **pandas**, **matplotlib**, and **seaborn** for data analysis & visualization


In [ ]:
import os
import requests
import json
from typing import Dict, List, Any
from crewai import Agent, Task, Crew, Process
from crewai_tools import FirecrawlSearchTool, SerperDevTool
from smolagents import CodeAgent, ToolCallingAgent, tool
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from dotenv import load_dotenv

load_dotenv()


## 🧰 Step 2: Define Custom Tools
We define helper tools (decorated with `@tool`) that agents can use:
1. **Sentiment Analysis** → Uses Hugging Face API  
2. **Entity Extraction** → Extracts entities (simulated here for demo)  
3. **Text Summarization** → Uses OpenAI GPT models  


In [ ]:
@tool
def analyze_sentiment(text: str) -> Dict[str, Any]:
    """Analyze sentiment using Hugging Face API"""
    API_URL = "https://api-inference.huggingface.co/models/distilbert-base-uncased-finetuned-sst-2-english"
    headers = {"Authorization": f"Bearer {os.getenv('HUGGINGFACE_API_KEY')}"}
    
    response = requests.post(API_URL, headers=headers, json={"inputs": text})
    if response.status_code == 200:
        return response.json()
    else:
        return {"error": f"API request failed: {response.status_code}"}


In [ ]:
@tool
def extract_entities(text: str) -> Dict[str, Any]:
    """Extract entities using spaCy API or similar service"""
    return {
        "entities": [
            {"text": "AI", "label": "TECHNOLOGY"},
            {"text": "Machine Learning", "label": "TECHNOLOGY"},
            {"text": "2024", "label": "DATE"}
        ]
    }


In [ ]:
@tool
def generate_summary(text: str) -> str:
    """Generate summary using OpenAI API"""
    OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
    if not OPENAI_API_KEY:
        return "OpenAI API key not configured"
    
    headers = {
        "Authorization": f"Bearer {OPENAI_API_KEY}",
        "Content-Type": "application/json"
    }
    
    data = {
        "model": "gpt-3.5-turbo",
        "messages": [
            {"role": "system", "content": "You are a helpful assistant that summarizes text."},
            {"role": "user", "content": f"Summarize this text in 3-5 bullet points: {text}"}
        ],
        "max_tokens": 200
    }
    
    response = requests.post("https://api.openai.com/v1/chat/completions", 
                           headers=headers, json=data)
    
    if response.status_code == 200:
        return response.json()['choices'][0]['message']['content']
    else:
        return f"Summary generation failed: {response.status_code}"


## 🤖 Step 3: Implement CrewAI System
The **CrewAI system** consists of:
- `Research Agent` → Performs web research  
- `Analysis Agent` → Runs sentiment analysis & entity extraction  
- `Report Agent` → Creates a polished research report  

We define a pipeline where tasks are executed **sequentially**.  


In [ ]:
class ResearchMultiAgentSystem:
    def __init__(self):
        self.search_tool = FirecrawlSearchTool()
        self.serper_tool = SerperDevTool()
        
        self.research_agent = Agent(
            role='Senior Research Analyst',
            goal='Conduct comprehensive research on given topics',
            backstory="""You are an expert researcher with 10+ years of experience
            in information gathering and analysis.""",
            tools=[self.search_tool, self.serper_tool],
            verbose=True,
            allow_delegation=False
        )
        
        self.analysis_agent = Agent(
            role='Data Analyst',
            goal='Analyze research data and extract meaningful insights',
            backstory="""You are a skilled data analyst who specializes in 
            extracting patterns and insights.""",
            tools=[analyze_sentiment, extract_entities],
            verbose=True,
            allow_delegation=False
        )
        
        self.report_agent = Agent(
            role='Technical Writer',
            goal='Create comprehensive research reports',
            backstory="""You are an experienced technical writer who excels at 
            creating clear, well-structured research reports.""",
            tools=[generate_summary],
            verbose=True,
            allow_delegation=False
        )
    
    def create_tasks(self, topic: str) -> List[Task]:
        return [
            Task(
                description=f"Research the topic '{topic}' comprehensively. Find at least 5 credible sources.",
                expected_output="Comprehensive research report",
                agent=self.research_agent
            ),
            Task(
                description="Analyze the research data to identify patterns and insights.",
                expected_output="Analysis report",
                agent=self.analysis_agent
            ),
            Task(
                description="Create a final comprehensive research report.",
                expected_output="Polished research report",
                agent=self.report_agent
            )
        ]
    
    def run_research(self, topic: str) -> str:
        crew = Crew(
            agents=[self.research_agent, self.analysis_agent, self.report_agent],
            tasks=self.create_tasks(topic),
            process=Process.sequential,
            verbose=True
        )
        result = crew.kickoff()
        return result


## 🤖 Step 4: Implement SmolAgents System
The **SmolAgents system** consists of:
- `Web Agent` → Fetches online research data  
- `Analysis Agent` → Performs data analysis (sentiment & entities)  
- `Manager Agent` → Coordinates other agents and compiles the report  

This is a **lightweight alternative** to CrewAI.  


In [ ]:
class SmolResearchSystem:
    def __init__(self):
        self.model = None  
        
        self.web_agent = ToolCallingAgent(
            tools=[FirecrawlSearchTool()],
            model=self.model,
            name="web_search_agent",
            description="Specializes in web research and information gathering"
        )
        
        self.analysis_agent = CodeAgent(
            tools=[analyze_sentiment, extract_entities],
            model=self.model,
            additional_authorized_imports=['pandas', 'numpy', 'matplotlib'],
            name="data_analysis_agent",
            description="Specializes in data analysis and insight extraction"
        )
        
        self.manager_agent = CodeAgent(
            tools=[generate_summary],
            model=self.model,
            managed_agents=[self.web_agent, self.analysis_agent],
            additional_authorized_imports=['json', 're'],
            name="research_manager",
            description="Coordinates research and analysis activities"
        )
    
    def run_research(self, topic: str) -> str:
        prompt = f"""
        Conduct comprehensive research on '{topic}'. Your team should:
        1. Search for current information and trends
        2. Analyze the data for insights and patterns
        3. Create a comprehensive summary report
        """
        return self.manager_agent.run(prompt)


## 🚀 Step 5: Run Example Research
Finally, we test both systems on the topic **"AI Agents in 2024"**.  
Each system should return a summarized research report.


In [ ]:
print("=== CrewAI Research System ===")
crewai_system = ResearchMultiAgentSystem()
crewai_result = crewai_system.run_research("AI Agents in 2024")
print("CrewAI Result:", crewai_result[:500] + "...")

print("\n=== SmolAgents Research System ===")
smol_system = SmolResearchSystem()
smol_result = smol_system.run_research("AI Agents in 2024")
print("SmolAgents Result:", smol_result[:500] + "...")
